# Pré-análise dos ficheiros

Analise do dataset **GDELT 2.0: Our Global World in Realtime**:
- Verificar o tamanho dos ficheiros
- Quantidade
- Datas
- Período abrangente

In [8]:
# Funções criadas para manipular os ficheiros S3
import funcoes_s3
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import pandas as pd

## Dados do S3 Bucket

https://registry.opendata.aws/gdelt/

In [9]:
# Configuração do S3
s3_resource = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
bucket_name = 'gdelt-open-data'

### Detalhes dos ficheiros

In [11]:
# Buscar todos os ficheiros da pasta "v2/gkg/"
list_filename = funcoes_s3.get_objects_name(s3_resource, bucket_name, 
                                       prefix="v2/gkg/")

In [12]:
list_filename.head()

,filename,size_b
0,v2/gkg/20150218230000.gkg.csv,33563207
1,v2/gkg/20150218231500.gkg.csv,31949650
2,v2/gkg/20150218233000.gkg.csv,35095453
3,v2/gkg/20150218234500.gkg.csv,34908148
4,v2/gkg/20150219000000.gkg.csv,30228677


## Análise geral

In [22]:
# Total de linhas
print(f"Quantidade de ficheiros: {list_filename['filename'].count()}")

Quantidade de ficheiros: 143435


In [18]:
size = pd.Series(list_filename['size_b'], name="size")
size.sample(5)

140462    17069936
6947      15154700
96135     16260655
100476    17503519
15467     45753240
Name: size, dtype: int64

In [24]:
print(f"Quantidade de linhas: {size.count()}")
print(f"Tamanho total do dataset (Gigabyte): {size.sum()/1024.0**3}")
print(f"Quantidade de valores nulos: {size[size.isnull()].count()}")

Quantidade de linhas: 143435
Tamanho total do dataset (Gigabyte): 4163.2315583229065
Quantidade de valores nulos: 0


### Agrupamento por ano

In [26]:
year = pd.Series(list_filename['filename'].str.slice(start=7, stop=11), name="year")
print(f"Quantidade de valores nulos: {year[year.isnull()].count()}")
summary = pd.concat([year, size], axis=1)
summary.sample(5)

Quantidade de valores nulos: 0


,year,size
141639,2019,34418247
47744,2016,25225760
83938,2017,37737
128912,2018,25993597
58783,2016,36783731


In [27]:
# Soma do tamanho dos ficheiros por ano
summary.groupby("year")["size"].sum() / 1024.0**3

year
2015     804.910736
2016    1186.154703
2017    1022.907219
2018     916.053842
2019     233.205058
Name: size, dtype: float64

In [28]:
# Quantidade de ficheiros por ano
summary.groupby("year")["size"].count()

year
2015    30107
2016    35090
2017    34049
2018    34048
2019    10141
Name: size, dtype: int64

### Agrupamento por ano e mês

In [29]:
year_month = pd.Series(list_filename['filename'].str.slice(start=7, stop=13), name="year_month")
print(f"Quantidade de valores nulos: {year_month[year_month.isnull()].count()}")
summary2 = pd.concat([year_month, size], axis=1)
summary2.sample(5)

Quantidade de valores nulos: 0


,year_month,size
34218,201602,44443381
112266,201805,37263832
56294,201609,31730873
4613,201504,20242136
33404,201602,33225113


In [30]:
summary2.groupby("year_month")["size"].sum() / 1024.0**3

year_month
201502     22.550392
201503     73.515927
201504     69.020093
201505     71.002230
201506     70.839559
201507     92.108864
201508     82.498530
201509     82.550022
201510     81.370623
201511     81.098707
201512     78.355789
201601     80.786035
201602     85.680989
201603     91.228398
201604     95.966824
201605    111.546812
201606    107.600369
201607    103.568461
201608    106.400418
201609    101.147179
201610    104.076129
201611    101.723961
201612     96.429128
201701    102.136230
201702     97.878540
201703    102.213628
201704     88.539621
201705     89.388645
201706     84.251412
201707     70.309095
201708     78.266292
201709     75.598354
201710     80.340855
201711     80.909507
201712     73.075040
201801     79.044870
201802     75.501170
201803     84.566465
201804     79.368310
201805     84.278939
201806     79.454609
201807     75.874994
201808     75.767649
201809     72.058220
201810     76.717828
201811     72.333935
201812     61.086852
20

In [31]:
summary2.groupby("year_month")["size"].count()

year_month
201502     954
201503    2974
201504    2878
201505    2940
201506    2880
201507    2975
201508    2975
201509    2880
201510    2807
201511    2876
201512    2968
201601    2968
201602    2784
201603    2973
201604    2875
201605    2971
201606    2872
201607    2972
201608    2976
201609    2876
201610    2975
201611    2873
201612    2975
201701    2976
201702    2688
201703    2953
201704    2843
201705    2907
201706    2797
201707    2829
201708    2903
201709    2800
201710    2914
201711    2748
201712    2691
201801    2751
201802    2524
201803    2831
201804    2718
201805    2853
201806    2783
201807    2913
201808    2966
201809    2880
201810    2976
201811    2880
201812    2973
201901    2976
201902    2688
201903    2975
201904    1502
Name: size, dtype: int64

### Agrupamento por ano, mês e dia

In [32]:
year_month_day = pd.Series(list_filename['filename'].str.slice(start=7, stop=15), name="year_month_day")
print(f"Quantidade de valores nulos: {year_month_day[year_month_day.isnull()].count()}")
summary3 = pd.concat([year_month_day, size], axis=1)
summary3.sample(5)

Quantidade de valores nulos: 0


,year_month_day,size
114674,20180619,45718267
141897,20190331,16222848
98904,20171228,26039020
140068,20190312,32242249
93597,20171029,24929926


In [33]:
summary3.groupby("year_month_day")["size"].sum() / 1024.0**3

year_month_day
20150218    0.126210
20150219    2.448347
20150220    2.461919
20150221    1.519326
20150222    1.386321
              ...   
20190412    2.630384
20190413    1.504289
20190414    1.342076
20190415    2.327859
20190416    1.443192
Name: size, Length: 1519, dtype: float64

In [34]:
summary3.groupby("year_month_day")["size"].count()

year_month_day
20150218     4
20150219    86
20150220    96
20150221    96
20150222    96
            ..
20190412    96
20190413    96
20190414    96
20190415    96
20190416    62
Name: size, Length: 1519, dtype: int64

## Exportar

In [40]:
prefix = "v2/gkg/"
summary4 = pd.concat([list_filename['filename'].str.replace(prefix, ""), 
                     list_filename['size_b'], 
                     pd.to_numeric(year), 
                     pd.to_numeric(year_month), 
                     pd.to_numeric(year_month_day)], axis=1)

summary4.sample(5)

,filename,size_b,year,year_month,year_month_day
78315,20170518154500.gkg.csv,58171507,2017,201705,20170518
91909,20171011221500.gkg.csv,59097451,2017,201710,20171011
23653,20151025153000.gkg.csv,22068253,2015,201510,20151025
44064,20160525140000.gkg.csv,52775658,2016,201605,20160525
77992,20170515053000.gkg.csv,29694266,2017,201705,20170515


In [53]:
! pwd

/mnt/c/Users/Rene Porto/OneDrive - ISCTE-IUL/Projecto Big Data


In [61]:
filepath = "/mnt/c/Users/Rene Porto/OneDrive - ISCTE-IUL/Projecto Big Data/filename.csv"
summary4.to_csv(filepath, index=False)

In [60]:
! ls -trla "/mnt/c/Users/Rene Porto/OneDrive - ISCTE-IUL/Projecto Big Data/filename.csv"

-rwxrwxrwx 1 rene rene 7594121 Apr 30 16:14 '/mnt/c/Users/Rene Porto/OneDrive - ISCTE-IUL/Projecto Big Data/Data/filename.csv'


## Resumo

Apesar do data set está a ser integrado a cada 15 minutos no repositório do projeto, os ficheiros que temos disponíveis são de notícias entre Fevereiro de 2015 e Abril de 2019, com um tamanho total de 4.163 Gygabytes.

Baseado nestas informações, iremos fazer o download dos ficheiros do dia 12 de abril de 2019 como exemplo e analisar os registos.